In [ ]:
import pickle
import pandas as pd
import numpy as np

In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -Vç

Python 3.9.19


In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
print("Q1: standard deviation of the predicted duration: ", np.std(y_pred).round(2))

Q1: standard deviation of the predicted duration:  6.25


In [9]:
year = 2023
month = 3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [10]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [11]:
df_result.to_parquet(
    './df_out.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [12]:
import os
file_size = os.path.getsize('./df_out.parquet')
file_size_kb = file_size / 1024
file_size_mb = file_size_kb / 1024
print("Q2: What's the size of the output file? ", round(file_size_mb))

Q2: What's the size of the output file?  65
